# L8b: Single Index Model Portfolio Optimization
In this lecture, we will explore the Single Index Model (SIM) and its application in portfolio optimization. The SIM simplifies the analysis of asset returns by assuming that the returns of individual assets are influenced by a single common factor, typically the market return. Here, we'll look at using single index models to describe portfiolios composed of only risky assets and a combination of risky and risk-free assets.

> __Learning Objectives:__
> 
> By the end of this lecture, you should be able to:
> 
> Three learning objectives goes here

Let's get started!
___

## Examples

> [▶ Let's estimate single index model parameter uncertainty from historical data](CHEME-5660-L7a-Example-SIM-Parameter-Uncertainty-Fall-2025.ipynb). In this example, we estimate single index models from historical data and quantify the uncertainty in our parameter estimates using bootstrap simulation.
___

## Concept Review: Single Index Model (SIM)
The Single Index Model (SIM) is a simplified approach to understanding the relationship between the returns of individual assets and the overall market. It assumes that the return of an asset can be expressed as a linear function of the market return plus a unique component specific to the asset.


> __Single Index Model (SIM)__
> 
> Single index models are factor models that consider only the return (growth) of the market factor. These models were originally developed by Sharpe, 1963: [Sharpe, William F. (1963). "A Simplified Model for Portfolio Analysis". Management Science, 9(2): 277-293. doi:10.1287/mnsc.9.2.277.](https://pubsonline.informs.org/doi/abs/10.1287/mnsc.9.2.277)
>
> Suppose the growth of firm $i$ at time $t$ is denoted by $g^{(t)}_{i}$. Then, the single index model of the return (growth rate) is given by:
> $$
g^{(t)}_{i} = \alpha_{i} + \beta_{i}\;g^{(t)}_{M} + \epsilon^{(t)}_{i}
$$
> where $\alpha_{i}$ is the _idiosyncratic (firm-specific) growth_, $\beta_{i}$ is the component of the growth rate of firm $i$ explained by the market (it is also a measure of risk), $g^{(t)}_{M}$ is the growth rate of the market index at time $t$, and $\epsilon^{(t)}_{i}$ denotes an error model associated with firm $i$ (describes growth rate not captured by the firm or market factors). 



### Simulation: Bootstrapping parameter uncertainty
Finally, an interesting way to explore parameter uncertainty is through a Monte Carlo simulation procedure called bootstrapping.

> __Bootstrapping__ The idea is to use our estimated parameters and error model to generate many __synthetic datasets__, then examine the empirical distribution of parameter estimates that we obtain from re-estimating parameters using these datasets. This approach is called __bootstrapping__ because we are _pulling ourselves up by our bootstraps_, i.e., using our fitted model to generate new data and re-estimate parameters, thereby empirically exploring the sampling distribution of our estimator. 

This allows us to see if the empirical results match our theoretical expectations. Let's look at some pseudocode for how we might implement this simulation.

__Initialization:__ Given the design matrix $\hat{\mathbf{X}}$, the estimated parameters $\hat{\boldsymbol{\theta}}$ and the error variance $\hat{\sigma}^2$ from our observed data, a value for the regularization parameter $\delta\geq{0}$, and the number of samples to generate $K$. 

For each $k = 1, 2, \ldots, K$: __do__:
1. Generate synthetic errors: $\boldsymbol{\varepsilon}^{(k)} \sim \mathcal{N}(\mathbf{0}, \Delta{t}\;\hat{\sigma}^2\;\mathbf{I})$
2. Create synthetic observations: $\mathbf{y}^{(k)} \gets \hat{\mathbf{X}}\;\hat{\boldsymbol{\theta}} + \boldsymbol{\varepsilon}^{(k)}$
3. Estimate parameters from the synthetic observation: $\hat{\boldsymbol{\theta}}^{(k)} \gets \left(\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}} + \delta\;\mathbf{I}\right)^{-1}\hat{\mathbf{X}}^{\top}\mathbf{y}^{(k)}$

Analyze the empirical distribution of $\left\{\hat{\boldsymbol{\theta}}^{(1)}, \hat{\boldsymbol{\theta}}^{(2)}, \ldots, \hat{\boldsymbol{\theta}}^{(K)}\right\}$. 

The empirical mean and covariance of the simulated parameter estimates should approximate our theoretical result:
$$
\begin{align*}
\text{Empirical mean} &\approx \mathbf{S}\;\hat{\boldsymbol{\theta}}\\
\text{Empirical covariance} &\approx \Delta{t}\;\hat{\sigma}^2\;\left(\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}} + \delta\;\mathbf{I}\right)^{-1}
\end{align*}
$$


Let's look at an example where we quantify parameter uncertainty using bootstrap simulation.

> __Example__
>
> [▶ Let's estimate single index model parameter uncertainty from historical data](CHEME-5660-L7a-Example-SIM-Parameter-Uncertainty-Fall-2025.ipynb). In this example, we estimate single index models from historical data and quantify the uncertainty in our parameter estimates using bootstrap simulation.

___

## Risky Asset Portfolios using Single Index Models
Fill me in.

## Risky and Risk-Free Asset Portfolios using Single Index Models
Fill me in.

## Summary
Single summary sentence goes here.

> __Key Takeaways:__
>
> Three key takeaways goes here

Single conclusion sentence goes here.
___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___